# Calculating Alternative Scoring Scenarios

The analysis below calculates the total points and final standings of the ice dance and men's competitions under two alternative scenarios:

1. The scores of two countries' judges were replaced with the "average" of the rest of the judging panel.
2. The scores of two countries' judges were removed from the judging panel entirely.

For ice dance, those countries are Canada and France.

For men's figure skating, those countries are the United States and China.

## Load scoring data

In [1]:
import pandas as pd

In [2]:
performances = pd.read_csv("../data/performances.csv")
aspects = pd.read_csv("../data/judged-aspects.csv")
scores = pd.read_csv("../data/judge-scores.csv")
judge_goe = pd.read_csv("../data/judge-goe.csv")

In [3]:
performances.groupby(["program", "competition"])\
    .size().to_frame("performances")

,,performances
program,competition,
Ice Dance - Free Dance,Olympic Winter Games 2018,20
Ice Dance - Short Dance,Olympic Winter Games 2018,24
Ladies Single Skating - Short Program,Olympic Winter Games 2018,30
Men Single Skating - Free Skating,Olympic Winter Games 2018,24
Men Single Skating - Short Program,Olympic Winter Games 2018,30
Pair Skating - Free Skating,Olympic Winter Games 2018,16
Pair Skating - Short Program,Olympic Winter Games 2018,22
Team Event - Ice Dance Free Dance,Olympic Winter Games 2018,5
Team Event - Ice Dance Short Dance,Olympic Winter Games 2018,10


In [4]:
scores_with_context = scores.pipe(
    pd.merge,
    aspects,
    on = "aspect_id",
    how = "left"
).pipe(
    pd.merge,
    performances,
    on = "performance_id",
    how = "left"
).pipe(
    pd.merge,
    judge_goe,
    on = [ "aspect_id", "judge" ],
    how = "left"
)

In [5]:
assert len(scores) == len(scores_with_context)

## Set up score-totaling calculations

In [6]:
# The ISU's scoring system uses a trimmed mean, which removes the highest and 
# lowest score for each element and component.
# To calculate the proper score for a competition, we do the same here.
def calc_trimmed_mean(score_list):
    trimmed = sorted(score_list)[1:-1]
    return round(sum(trimmed) / len(trimmed), 2)

In [7]:
def calculate_aspect_scores(aspect_judgments):

    aspects_grp = aspect_judgments.groupby("aspect_id")
    
    scores = pd.DataFrame({
        "name": aspects_grp["name"].first(),
        "program": aspects_grp["program"].first(),
        "section": aspects_grp["section"].first(),
        "performance_id": aspects_grp["performance_id"].first(),
        "factor": aspects_grp["factor"].first(),
        "base_value": aspects_grp["base_value"].first(),
        "score": aspects_grp["score"].apply(lambda x: calc_trimmed_mean(x)),
        "judge_goe": aspects_grp["judge_goe"].apply(lambda x: calc_trimmed_mean(x)),
        "total_deductions": aspects_grp["total_deductions"].first()
    })
    
    return scores

In [8]:
def total_points(row):
    if row["section"] == "elements":
        return round(row["base_value"] + row["judge_goe"], 2)
    
    elif row["section"] == "components":
        return round(row["factor"] * row["score"], 2)
    
    else:
        print("Unknown section: {}".format(row["section"]))
        return None

In [9]:
def calculate_results(scores):
    
    scores["total_points"] = scores.apply(total_points, axis=1)
    
    perfs_grp = scores.groupby("performance_id")
    
    perfs = pd.DataFrame({
        "score": perfs_grp["total_points"].sum(),
        "deductions": perfs_grp["total_deductions"].first(),
        "program": perfs_grp["program"].first(),
        "name": perfs_grp["name"].first()
    })
    
    perfs["total_score"] = perfs["score"] - perfs["deductions"]
    
    comp_grp = perfs.groupby("name")
    
    results = pd.DataFrame({
        "final_score": comp_grp["total_score"].sum()
     })
    
    return results

In [10]:
def calc_trimmed_mean_with_average_judge(score_list):
    average = sum(score_list) / len(score_list)
    score_list = score_list.tolist() + [average] * (9-len(score_list))
    trimmed = sorted(score_list)[1:-1]
    return round(sum(trimmed) / len(trimmed), 2)

In [11]:
def calculate_score_with_average_judge(dataframe):

    aspects = dataframe.groupby("aspect_id")
    
    scores = pd.DataFrame({
        "name": aspects["name"].first(),
        "program": aspects["program"].first(),
        "section": aspects["section"].first(),
        "performance_id": aspects["performance_id"].first(),
        "factor": aspects["factor"].first(),
        "base_value": aspects["base_value"].first(),
        "scores_of_panel": aspects["scores_of_panel"].first(),
        "score": aspects["score"].apply(lambda x: calc_trimmed_mean_with_average_judge(x)),
        "judge_goe": aspects["judge_goe"].apply(lambda x: calc_trimmed_mean_with_average_judge(x)),
        "total_deductions": aspects["total_deductions"].first()
    })
    
    return scores

---

## Ice Dance Calculations

### Create subsets of scores with/without Canadian and French judges

In [12]:
id_olympics = scores_with_context[
    scores_with_context["program"].str.contains("Ice Dance") &
    ~scores_with_context["program"].str.contains("Team")
].copy()

In [13]:
id_olympics\
    .groupby([ "program", "judge"])\
    .size().unstack()\
    .fillna(0).astype(int)

judge,J1,J2,J3,J4,J5,J6,J7,J8,J9
program,,,,,,,,,
Ice Dance - Free Dance,277,277,277,277,277,277,277,277,277
Ice Dance - Short Dance,240,240,240,240,240,240,240,240,240


In [14]:
id_olympics_without_canada_france = id_olympics[
    ~(
        # Canada
        (
            (id_olympics["judge"] == "J1") &
            (id_olympics["program"] == "Ice Dance - Short Dance")
        ) |
        (
            (id_olympics["judge"] == "J2") &
            (id_olympics["program"] == "Ice Dance - Free Dance")
        ) |
      
        # France
        (

            (id_olympics["judge"] == "J4") &
            (id_olympics["program"] == "Ice Dance - Short Dance")
        )
    )
].copy()

In [15]:
id_olympics_without_canada_france\
    .groupby([ "program", "judge"])\
    .size().unstack()\
    .fillna(0).astype(int)

judge,J1,J2,J3,J4,J5,J6,J7,J8,J9
program,,,,,,,,,
Ice Dance - Free Dance,277,0,277,277,277,277,277,277,277
Ice Dance - Short Dance,0,240,240,0,240,240,240,240,240


### Calculate actual and alternate scoring scenarios for Ice Dance

In [16]:
aspect_scores = calculate_aspect_scores(id_olympics)
final_actual = calculate_results(aspect_scores)\
    .sort_values("final_score", ascending=False)

In [17]:
scores_with_replacement = calculate_score_with_average_judge(id_olympics_without_canada_france)
final_with_replacement = calculate_results(scores_with_replacement)\
    .sort_values("final_score", ascending=False)

In [18]:
scores_no_replacement = calculate_aspect_scores(id_olympics_without_canada_france)
final_with_removal = calculate_results(scores_no_replacement)\
    .sort_values("final_score", ascending=False)

### Actual scores vs. scores with CAN/FRA replaced by "average" judgments

In [19]:
pd.merge(
    final_actual.reset_index(),
    final_with_replacement.reset_index(),
    on="name",
    suffixes=["_actual", "_with_replacement"]
).set_index("name")\
    .assign(
        rank_actual = lambda x: x["final_score_actual"]\
            .rank(ascending = False)\
            .astype(int),
        rank_with_replacement = lambda x: x["final_score_with_replacement"]\
            .rank(ascending = False)\
            .astype(int),
    )

,final_score_actual,final_score_with_replacement,rank_actual,rank_with_replacement
name,,,,
VIRTUE Tessa / MOIR Scott,206.07,205.74,1,2
PAPADAKIS Gabriella / CIZERON Guillaume,205.28,206.13,2,1
SHIBUTANI Maia / SHIBUTANI Alex,192.59,192.27,3,3
HUBBELL Madison / DONOHUE Zachary,189.69,189.48,4,4
BOBROVA Ekaterina / SOLOVIEV Dmitri,186.92,187.76,5,5
CAPPELLINI Anna / LANOTTE Luca,186.91,186.99,6,6
WEAVER Kaitlyn / POJE Andrew,181.98,180.31,7,7
CHOCK Madison / BATES Evan,179.58,178.95,8,8
GILLES Piper / POIRIER Paul,176.91,176.18,9,9


### Actual scores vs. scores with CAN/FRA removed (and not replaced)

In [20]:
pd.merge(
    final_actual.reset_index(),
    final_with_removal.reset_index(),
    on="name",
    suffixes=["_actual", "_with_removal"]
).set_index("name")\
    .assign(
        rank_actual = lambda x: x["final_score_actual"]\
            .rank(ascending = False)\
            .astype(int),
        rank_with_removal = lambda x: x["final_score_with_removal"]\
            .rank(ascending = False)\
            .astype(int),
    )

,final_score_actual,final_score_with_removal,rank_actual,rank_with_removal
name,,,,
VIRTUE Tessa / MOIR Scott,206.07,205.86,1,2
PAPADAKIS Gabriella / CIZERON Guillaume,205.28,206.32,2,1
SHIBUTANI Maia / SHIBUTANI Alex,192.59,192.27,3,3
HUBBELL Madison / DONOHUE Zachary,189.69,189.42,4,4
BOBROVA Ekaterina / SOLOVIEV Dmitri,186.92,187.97,5,5
CAPPELLINI Anna / LANOTTE Luca,186.91,187.00,6,6
WEAVER Kaitlyn / POJE Andrew,181.98,180.34,7,7
CHOCK Madison / BATES Evan,179.58,178.83,8,8
GILLES Piper / POIRIER Paul,176.91,176.22,9,9


---

## Men's Figure Skating Calculations

### Create subsets of scores with/without American and Chinese judges

In [21]:
men_olympics = scores_with_context[
    scores_with_context["program"].str.contains("Men") &
    ~scores_with_context["program"].str.contains("Team")
].copy()

In [22]:
men_olympics\
    .groupby([ "program", "judge"])\
    .size().unstack()\
    .fillna(0).astype(int)

judge,J1,J2,J3,J4,J5,J6,J7,J8,J9
program,,,,,,,,,
Men Single Skating - Free Skating,432,432,432,432,432,432,432,432,432
Men Single Skating - Short Program,358,358,358,358,358,358,358,358,358


In [23]:
men_olympics_without_usa_china = men_olympics[
    ~(
        # China
        (
            (men_olympics["judge"] == "J9") &
            (men_olympics["program"] == "Men Single Skating - Short Program")
        ) |
        (
            (men_olympics["judge"] == "J7") &
            (men_olympics["program"] == "Men Single Skating - Free Skating")
        ) |
      
        # United States
        (

            (men_olympics["judge"] == "J2") &
            (men_olympics["program"] == "Men Single Skating - Free Skating")
        )
    )
].copy()

In [24]:
men_olympics_without_usa_china\
    .groupby([ "program", "judge"])\
    .size().unstack()\
    .fillna(0).astype(int)

judge,J1,J2,J3,J4,J5,J6,J7,J8,J9
program,,,,,,,,,
Men Single Skating - Free Skating,432,0,432,432,432,432,0,432,432
Men Single Skating - Short Program,358,358,358,358,358,358,358,358,0


### Calculate actual and alternate scoring scenarios for Men

In [25]:
men_aspect_scores = calculate_aspect_scores(men_olympics)
men_final_actual = calculate_results(men_aspect_scores)\
    .sort_values("final_score", ascending=False)

In [26]:
men_scores_with_replacement = calculate_score_with_average_judge(men_olympics_without_usa_china)
men_final_with_replacement = calculate_results(men_scores_with_replacement)\
    .sort_values("final_score", ascending=False)

In [27]:
men_scores_no_replacement = calculate_aspect_scores(men_olympics_without_usa_china)
men_final_with_removal = calculate_results(men_scores_no_replacement)\
    .sort_values("final_score", ascending=False)

### Actual scores vs. scores with CHN/USA replaced by "average" judgments

In [28]:
pd.merge(
    men_final_actual.reset_index(),
    men_final_with_replacement.reset_index(),
    on="name",
    suffixes=["_actual", "_with_replacement"]
).set_index("name")\
    .assign(
        rank_actual = lambda x: x["final_score_actual"]\
            .rank(ascending = False)\
            .astype(int),
        rank_with_replacement = lambda x: x["final_score_with_replacement"]\
            .rank(ascending = False)\
            .astype(int),
    )

,final_score_actual,final_score_with_replacement,rank_actual,rank_with_replacement
name,,,,
HANYU Yuzuru,317.85,318.68,1,1
UNO Shoma,308.90,310.34,2,2
FERNANDEZ Javier,305.24,307.01,3,3
JIN Boyang,299.77,298.09,4,5
CHEN Nathan,299.35,298.73,5,4
ZHOU Vincent,276.69,275.75,6,6
ALIEV Dmitri,271.51,272.60,7,7
KOLYADA Mikhail,270.25,271.83,8,8
CHAN Patrick,265.43,264.71,9,9


### Actual scores vs. scores with CHN/USA removed (and not replaced)

In [29]:
pd.merge(
    men_final_actual.reset_index(),
    men_final_with_removal.reset_index(),
    on="name",
    suffixes=["_actual", "_with_removal"]
).set_index("name")\
    .assign(
        rank_actual = lambda x: x["final_score_actual"]\
            .rank(ascending = False)\
            .astype(int),
        rank_with_removal = lambda x: x["final_score_with_removal"]\
            .rank(ascending = False)\
            .astype(int),
    )

,final_score_actual,final_score_with_removal,rank_actual,rank_with_removal
name,,,,
HANYU Yuzuru,317.85,318.80,1,1
UNO Shoma,308.90,310.44,2,2
FERNANDEZ Javier,305.24,307.14,3,3
JIN Boyang,299.77,298.15,4,5
CHEN Nathan,299.35,298.82,5,4
ZHOU Vincent,276.69,275.80,6,6
ALIEV Dmitri,271.51,272.63,7,7
KOLYADA Mikhail,270.25,271.92,8,8
CHAN Patrick,265.43,264.67,9,9


---

---

---